In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta
from hashlib import sha256
import codecs


from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer
from bulletin.commom import static
from bulletin.commom.normalize import *

pd.set_option('display.max_columns', None)

output = join("output","correcoes","casos_confirmados")

if not isdir(output):
    makedirs(output)

timer = Timer()

In [2]:
timer.reset()
casos_confirmados = CasosConfirmados()
casos_confirmados.update()
timer.stop()

KeyboardInterrupt: 

In [5]:
# casos = casos_confirmados.fix_mun_resid_casos('mun_atend')
# obitos = casos_confirmados.fix_mun_resid_obitos()
casos = casos_confirmados.get_casos()
casos = casos.drop(columns=['obito','data_obito','excluir'])
obitos = casos_confirmados.get_obitos()

print(f"{len(casos)};{len(obitos)}")
obitos.loc[~obitos['hash'].isin(casos['hash'])]

425374;8155


,ordem,ibge7,nome,sexo,idade,municipio,rs,data_do_obito,data_comunicacao_obito,hash,hash_less,hash_more
35,36,4101507,DARCY SEGUNDINO VENDRAMETTO,M,80,ARAPONGAS,16,2020-04-13,2020-04-13,a15c8f8f197d352b6eac190488a1d15111d4d1d0bad6c0...,f35d863c1b8e9bd9de4228cf43cf004f603b519a4f47c6...,4e3d0bc9efb935be41cec27c9aac614659338647e77db5...
121,122,4125506,IVO DA SILVA,M,58,SAO JOSE DOS PINHAIS,02,2020-05-10,2020-05-10,3991e73c627a9c2d9733712d3c165c70d2e4abd741f673...,1e7c3b497de76e3d3197b4abedcd4e1477b27c13a74f40...,e80a92ec671a99460e206c265a18632675836daeb40c4f...
138,139,4125506,APARECIDA GOMES DE MORAIS,F,76,SAO JOSE DOS PINHAIS,02,2020-05-17,2020-05-17,313650146b427513f8bc41be2868fc69e2e7b9233475e5...,c355f2e582d11373a325a1283a6fceed26f60c74e135a7...,a2d155d27985674063b30e6d8429c6292033a3b6cad637...
141,142,4104808,LUIZ AUGUSTO DOS SANTOS,M,86,CASCAVEL,10,2020-05-17,2020-05-17,7eb71abeceff1162a3ffde053469c02e3c8aceea703613...,0e0c629e8da27cdf6099685b61ca8c2dc747e948c77da5...,950990602728837e8d8d81f83f36439ae91b62cfba3fb8...
144,145,4118402,JOSE PAULO DIAS DA SILVA,M,54,PARANAVAI,14,2020-05-18,2020-05-18,ec6c8e5594b396d402ccbca1622a670ba9f7be567c81e8...,b320ed171c02ee86fa6f052f13c4917ba27cf47158ae87...,07677d0055c3eaae3bea4e4866703fb1fe8a22786d7307...
...,...,...,...,...,...,...,...,...,...,...,...,...
5244,5245,4119905,CELSO CERQUEIRA LIMA,M,56,PONTA GROSSA,03,2020-10-15,2020-10-15,e99244b09d39fa9aa7da7e00c353992738e4b06bb22a7e...,62d07c867784ed4cc1f28f579dc1c4bafb74e91a924c81...,f865d08b26ce6122c040d0838e261769d892f24305b47c...
5876,5877,4104808,JOSE ROBERTO MOREIRA,M,69,CASCAVEL,10,2020-11-13,2020-11-13,be11790a4c5fd810efc513d5d9e7a0b7dfab607aeab6ea...,0dea58202456a05b9b0d587d67e15be3a79f16c5b5b89b...,b254060c26304bc3262110ad799a37a54d398588794240...
5966,5967,4105805,SELMA NUNES SCHMIDT,F,72,COLOMBO,02,2020-11-16,2020-11-16,5ded8218caed084150c747ca250e6b73b976919f945864...,2f9fef0bf77c03e79f0e351afd4e10b8ab4caf44801564...,b817b9f64b56e7b8e01d9562875f02cf57800b610a38e1...
6143,6144,4104808,SARA DE SOUZA PADILHA,F,64,CASCAVEL,10,2020-11-23,2020-11-25,654e91ee7f226934e5ab8f657130a0f56e8c2612de293c...,32eb43614b559bbc653efa263c1fa9801e09129f548606...,68289e7664494e54214fc3bba8b660d1a20624c7d898ab...


In [4]:
# exit()
# excluir.to_excel('duplicados_exclusao.xlsx')
# today = datetime.today()

# with codecs.open(join('output','relatorios',f"relatorio_exclusao.txt"),"w","utf-8-sig") as relatorio:
#     relatorio.write(f"Alterações {today.strftime('%d/%m/%Y')}\n")
#     relatorio.write(f"Foram excluido {len(excluir)} casos duplicados no decorrer de 2020\n\n")

#     relatorio.write(f"Por municipio foram:\n")
#     for mun, df in excluir.groupby('mun_resid'):
#         relatorio.write(f"{mun}\t{len(df)}\n")

#     relatorio.write(f"Por data de divulgação foram:\n")
#     for data, df in excluir.groupby('dt_diag'):
#         relatorio.write(f"{data.strftime('%d/%m/%Y')}\t{len(df)}\n")

#     relatorio.write(f"\n\n")
#     for idx, row in excluir.iterrows():
#         relatorio.write(f"Um caso confirmado ({row['sexo'],row['idade']}) no dia {row['dt_diag'].dt.strftime('%d/%m/%Y')} em {row['mun_resid']} foi excluido por duplicidade.\n")



SystemExit: 

In [ ]:
obitos = obitos.rename(columns={'data_do_obito': 'data_obito'})
obitos['obito'] = 'SIM'

casos = pd.merge(left=casos, right=obitos[['obito','data_obito','hash']], on='hash', how='left')

casos.loc[casos['obito'].isnull(), 'obito'] = 'NAO'
casos.loc[(casos['obito']=="SIM") & (casos.duplicated('hash')), 'obito'] = 'NAO'
casos.loc[(casos['obito']=="SIM") & (casos.duplicated('hash')), 'data_obito'] = pd.NaT

print(f"{len(casos)};{len(casos.loc[casos['obito']=='SIM'])}")

In [ ]:
casos['excluir'] = 'NAO'

print(f"Duplicados com a mesma idade: {casos.loc[casos.duplicated('hash', keep=False)].shape}")
print(f"Duplicados com idade menos 1: {casos.loc[casos['hash_less'].isin(casos['hash'])].shape}")
casos.loc[casos['hash_less'].isin(casos['hash']), 'hash'] = casos.loc[casos['hash_less'].isin(casos['hash']), 'hash_less']
print(f"Duplicados com idade mais 1: {casos.loc[casos['hash_more'].isin(casos['hash'])].shape}")
casos.loc[casos['hash_more'].isin(casos['hash']), 'hash'] = casos.loc[casos['hash_more'].isin(casos['hash']), 'hash_more']
print(f"todos os duplicados: {casos.loc[casos.duplicated('hash', keep=False)].shape}")
duplicados = casos.loc[casos.duplicated('hash', keep=False)]
print(f"Valores unicos: {len(duplicados.groupby('hash'))}")

In [ ]:
def get_better(cc):
    scores = np.zeros(len(cc))

    for i, (_, cc_row) in enumerate(cc.iterrows()):

        if cc_row['obito'] == 'SIM':
            scores[i] = 1

    i = np.argmax(scores)
    return cc.iloc[i].name

In [ ]:
manter = []
for cc_group in duplicados.groupby('hash'):
    
    idx = get_better(cc_group[1])
    manter.append(idx)

In [ ]:
duplicados = set(duplicados.index.tolist())
duplicados = duplicados - set(manter)

casos.loc[duplicados,'excluir'] = 'SIM'
casos.loc[duplicados,'hash'] = None
casos.loc[duplicados,'hash_less'] = None
casos.loc[duplicados,'hash_more'] = None

In [ ]:
print(casos.loc[casos['excluir']=='SIM'].shape)

In [ ]:
new_casos = casos[['ordem','ibge7','nome','sexo','idade','mun_resid','mun_atend','rs','laboratorio','dt_diag','comunicacao','is','obito','data_obito','excluir']]
new_obitos = obitos[['ordem','ibge7','nome','sexo','idade','municipio','rs','data_obito']]

In [ ]:
writer = pd.ExcelWriter(join(output,"Casos confirmados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

new_casos.to_excel(writer,sheet_name='Casos confirmados',index=None)
new_obitos.to_excel(writer,sheet_name='Obitos',index=None)

writer.save()
writer.close()

In [ ]:
exit()
# new_casos = casos[['ordem','ibge7','nome','sexo','idade','mun_resid','mun_atend','rs','laboratorio','dt_diag','comunicacao','is']]
for grupo_casos, grupo_obitos in zip(new_casos.groupby('rs'),new_obitos.groupby('rs')):
    writer = pd.ExcelWriter(join(output,f"casos_confirmados_{int(grupo_casos[0])}a_regional.xlsx"),
                            engine='xlsxwriter',
                            datetime_format='dd/mm/yyyy',
                            date_format='dd/mm/yyyy')

    grupo_casos[1].to_excel(writer,sheet_name='Casos confirmados',index=None)
    grupo_obitos[1].to_excel(writer,sheet_name='Obitos',index=None)

    writer.save()
    writer.close()

In [ ]:
writer = pd.ExcelWriter(join(output,f"casos_confirmados_nao_residentes.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

new_casos.loc[new_casos['rs'].isnull()].to_excel(writer,sheet_name='Casos confirmados',index=None)
new_obitos.loc[new_obitos['rs'].isnull()].to_excel(writer,sheet_name='Obitos',index=None)

writer.save()
writer.close()

In [ ]:
casosc['nome_hash'] = casosc['nome'].apply(normalize_hash)
casost['nome_hash'] = casost['nome'].apply(normalize_hash)
casosc = casosc.sort_values('nome_hash')
casost = casost.sort_values('nome_hash')
all(casosc['nome_hash'].values == casost['nome_hash'].values)
casost.loc[casost['status'].notnull(),'status'] = 'RECUPERADO'